In [1]:
import os
os.environ['HF_HOME'] = '/data/loriss21dm/cache'
from datasets import load_dataset

In [2]:
dataset_name = "loris3/stratified_10m_curriculum"

In [3]:
dataset = load_dataset(dataset_name)

In [4]:
df = dataset["train"].to_pandas()

In [5]:
df

,text,source,stage
0,huh .,aochildes.txt,C1: Child Directed Speech
1,michael .,aochildes.txt,C1: Child Directed Speech
2,done !,aochildes.txt,C1: Child Directed Speech
3,you wanna put that in there okay don't knock i...,aochildes.txt,C1: Child Directed Speech
4,the female is half brown ?,aochildes.txt,C1: Child Directed Speech
...,...,...,...
1070316,suppose the sun to be somewhere. If her Majest...,gutenberg.txt,C5: Written English
1070317,"reputed to be) that of a Mrs. Leakey, an amiab...",gutenberg.txt,C5: Written English
1070318,the people. Hokosa was the accuser. In brief a...,gutenberg.txt,C5: Written English
1070319,himself. One thing and one alone had developed...,gutenberg.txt,C5: Written English


In [15]:
import json
import pandas as pd
from huggingface_hub import hf_hub_download
info = None
with open(hf_hub_download(repo_id=dataset_name, filename="info.json", repo_type="dataset")) as f:
    info = json.load(f)

info["curriculum"]

{'C1: Child Directed Speech': ['aochildes.txt'],
 "C2: Children's Books": ['children_stories.txt',
  'cbt_test.txt',
  'cbt_train.txt',
  'cbt_valid.txt'],
 'C3: Dialogue': ['switchboard.txt', 'bnc_spoken.txt', 'open_subtitles.txt'],
 'C4: Educational': ['qed.txt', 'simple_wiki.txt'],
 'C5: Written English': ['wikipedia.txt', 'gutenberg.txt']}

In [8]:
df["word_count"] = df["text"].apply(lambda d: len(d.split()))

In [28]:
counts = df.groupby(by="stage")["stage"].count()
counts.name = "Documents"

In [33]:
counts_proportion = df.groupby(by="stage")["stage"].count() / df.groupby(by="stage")["stage"].count().sum()
counts_proportion = counts_proportion.map(lambda n: '{:,.2%}'.format(n))
counts_proportion.name = ""

In [34]:
words = df.groupby(by="stage")["word_count"].sum()
words.name = "Words"

In [37]:
words_proportion = df.groupby(by="stage")["word_count"].sum() / df["word_count"].sum()
words_proportion = words_proportion.map(lambda n: '{:,.2%}'.format(n))
words_proportion.name = ""

In [47]:
stats = pd.concat([words,words_proportion, counts, counts_proportion], axis=1)
stats.index.name = "Stage"
stats

,Words,,Documents,
Stage,,,,
C1: Child Directed Speech,1999999,20.00%,360533,33.68%
C2: Children's Books,1999995,20.00%,77384,7.23%
C3: Dialogue,1999987,20.00%,349650,32.67%
C4: Educational,1999999,20.00%,161554,15.09%
C5: Written English,1999945,20.00%,121200,11.32%
